In [11]:
from aiida import load_profile, common, orm 
load_profile();

import ipywidgets as widgets
from IPython.display import display
from widgets import query, misc
import aiidalab_widgets_base as awb
from settings import *
import pandas as pd
import base64
from utils import utils
from pathlib import Path
from IPython.core.display import display, HTML

display(HTML("""<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>"""))

path_config = Path.cwd()/'config/stash_files.yaml'

# FLEXPART-COSMO/IFS results
## and sensitivities collection job

How it works:<br>
<ul>
    <li> <b> 🔍 Step 1:</b> Query for the transport model results. You can search dirctly by presetting in the basic tab below (recommended),<br> or enter in the advance tab and select each paramter individualy. </li>
    <li> <b> 🚀 Step 2:</b> Select the desired results and click submit to start the collect sensitivities job. </li>
</ul>

<b>Surface footprints</b> can be seen by clicking on the specific day (if available).

In [12]:
misc = misc.Misc()
q = query.SearchCalculations()
q

SearchCalculations(children=(HBox(children=(Text(value='2020-01-01--2020-12-31', description='range_dates', pl…

In [ ]:
def fill_df():
    """ Reformat and filter q.remotes
        to create q.full_remotes dataframe
        containing the selected results only
     """
    for i in q.w_checkboxes:
        if i.value == True:
            group_name = i.description
            q.full_remotes = pd.concat([q.full_remotes,q.remotes.loc[q.remotes['w_hash'] == group_name]],
            ignore_index = True)
    if q.full_remotes.empty:
        return 
    else:
        q.full_remotes['date'] = q.full_remotes['date'].str[:10]
        q.full_remotes['id'] = q.full_remotes['date'].replace('-','_', regex=True)+q.full_remotes['location'].replace('-','_', regex=True)


In [ ]:
def get_file_to_download(change=None):
        #Creates df of selected results to be downloaded
        fill_df()
        if not q.full_remotes.empty:
                q.full_remotes['post_stash_address']=[i.target_basepath for i in q.full_remotes.RemoteStash]
                q.full_remotes['flexpart_stash_address']=[i.target_basepath for i in q.full_remotes.flexpart_stash]
                download_html.value = utils.download_button('flexpart_results.csv',
                                                        q.full_remotes[['date','flexpart_stash_address']],
                                                        'Download .csv flexpart results'
                                                        )
                download_html.value += utils.download_button('post_results.csv',
                                                        q.full_remotes[['date','post_stash_address']],
                                                        'Download .csv post results'
                                                        )
        
download_html = widgets.HTML('')

q.check_all.observe(get_file_to_download,names="value")
#TODO trigger by ? 
download_html

### Submit sensitivities collection

In [ ]:
out = widgets.Output()
computer = orm.load_computer(misc.computer.value)
prepend_text_ = f'#SBATCH --constrain=mc\n'+\
                f'#SBATCH --account=em05\n'

def prepare_post():
    fill_df()
    username = computer.get_configuration()['username']
    builder = COLLECT_SENS.get_builder()
    builder.code = orm.load_code(misc.collect_sens_code.value)
    builder.metadata.options.custom_scheduler_commands = prepend_text_
    builder.remote = dict(zip(q.full_remotes.id,q.full_remotes.RemoteStash))
    builder.model = q.full_remotes['model'][0]
    builder.outgrid = q.full_remotes['outgrid'][0]
    builder.outgrid_n = False
    if 'outgrid_n' in q.full_remotes.keys():
        builder.outgrid_n = True
    builder.metadata.options.stash = {
        'source_list':utils.read_yaml_data(path_config,
                                           names ='COLLECTION')['COLLECTION'],
        'target_base': f'/store/empa/em05/{username}/aiida_stash', #TODO
        'stash_mode': common.StashMode.COPY.value,
        }
    return builder

In [ ]:
btn_submit_ = awb.SubmitButtonWidget(
        COLLECT_SENS,
        inputs_generator = prepare_post,
        disable_after_submit = False,
        append_output = True,
    )
btn_submit_.btn_submit.button_style="success"
display(btn_submit_, out)